# Air Quality Prediction Experiment

This notebook combines PWWB, AirNow, and HRRR datasets for air quality prediction using TF-Keras.

# Experiment Configuration

In [1]:
# ========== EXPERIMENT CONFIGURATION ==========
# Change these variables to configure your experiment

EXPERIMENT_NAME = "January 2025 Fire Dataset"
EXPERIMENT_DESCRIPTION = "January 2025 fire period with PWWB, Airnow, and HRRR data."
EXPERIMENT_ID = "jan_2025_with_hrrr"

# Data parameters
START_DATE = "2025-01-01-00"
END_DATE = "2025-02-01-00"
TRAIN_SPLIT = 0.70  # 70% for training, 10% for val (will split later), leaving 20% for test.

# Model parameters
EPOCHS = 100
BATCH_SIZE = 16
WEIGHT_DECAY = 0.01

print(f"Experiment: {EXPERIMENT_NAME}")
print(f"Description: {EXPERIMENT_DESCRIPTION}")
print(f"Date Range: {START_DATE} to {END_DATE}")
print(f"Train/Test Split: {TRAIN_SPLIT*100:.0f}%/{(1-TRAIN_SPLIT)*100:.0f}% (temporal)")

Experiment: January 2025 Fire Dataset
Description: January 2025 fire period with PWWB, Airnow, and HRRR data.
Date Range: 2025-01-01-00 to 2025-02-01-00
Train/Test Split: 70%/30% (temporal)


# Data Parameters

In [2]:
# Define bounding box
lat_bottom, lat_top = 33.5, 34.5
lon_bottom, lon_top = -118.75, -117.0
extent = (lon_bottom, lon_top, lat_bottom, lat_top)

# Input data shape
dim = 40
frames_per_sample = 5

print(f"{extent}")
print(f"Grid dimension: {dim}x{dim}")
print(f"Frames per sample: {frames_per_sample}")

(-118.75, -117.0, 33.5, 34.5)
Grid dimension: 40x40
Frames per sample: 5


# Data Ingestion and Preprocessing

In [3]:
# Python nonsense that allows you to import from sibling directories
import sys
sys.path.append("../..")

import pandas as pd
from dotenv import load_dotenv

# Import the new PWWB implementation and dataset manager
from libs.pwwb import PWWBData
from libs.pwwb.utils.dataset_manager import create_dataset_manager

# Import the AirNow data class
from libs.airnowdata import AirNowData
from libs.hrrrdata import HRRRData

# Load environment variables (API keys, credentials)
load_dotenv()

import numpy as np
# Temporal train-test split function
def temporal_train_test_split(X, Y, train_size=0.8):
    """Split data temporally - first portion for training, last portion for testing"""
    split_idx = int(X.shape[0] * train_size)
    
    X_train = X[:split_idx]
    X_test = X[split_idx:]
    Y_train = Y[:split_idx]
    Y_test = Y[split_idx:]
    
    print(f"Temporal split at index {split_idx}:")
    print(f"  Training: samples 0-{split_idx-1} ({train_size*100:.0f}% of time)")
    print(f"  Testing: samples {split_idx}-{len(X)-1} ({(1-train_size)*100:.0f}% of time)")
    
    return X_train, X_test, Y_train, Y_test
    
def temporal_train_test_valid_split(X, Y, train_size=0.7, test_size=0.2, valid_size=0.1):
    train_split = int(X.shape[0] * train_size)
    test_split = int(X.shape[0] * (train_size + valid_size))
    
    X_train = X[:train_split]
    X_valid = X[train_split:test_split]
    X_test = X[test_split:]
    Y_train = Y[:train_split]
    Y_valid = Y[train_split:test_split]
    Y_test = Y[test_split:]
    
    print(f"Temporal split at indices {train_split} and {test_split}:")
    print(f"  Training: samples 0-{train_split-1} ({train_size*100:.0f}% of time)")
    print(f"  Validation: samples {train_split}-{test_split-1} ({valid_size*100:.0f}% of time)")
    print(f"  Testing: samples {test_split}-{len(X)-1} ({test_size*100:.0f}% of time)")
    
    return X_train, X_test, X_valid, Y_train, Y_test, Y_valid

# scale training data, then scale test data based on training data stats
import joblib
from sklearn.preprocessing import StandardScaler
def std_scale(X_train, X_test):
    scaler = StandardScaler()
    scaled_train = scaler.fit_transform(X_train.reshape(-1, 1)).reshape(X_train.shape)
    scaled_test = scaler.transform(X_test.reshape(-1, 1)).reshape(X_test.shape)

    return scaled_train, scaled_test

def std_scale_train_test_valid(X_train, X_test, X_valid):
    scaler = StandardScaler()
    scaled_train = scaler.fit_transform(X_train.reshape(-1, 1)).reshape(X_train.shape)
    scaled_test = scaler.transform(X_test.reshape(-1, 1)).reshape(X_test.shape)
    scaled_valid = scaler.transform(X_valid.reshape(-1, 1)).reshape(X_valid.shape)

    return scaled_train, scaled_test, scaled_valid

def scale(X, scaler_path):
    scaler = joblib.load(scaler_path)
    return scaler.transform(X.reshape(-1, 1)).reshape(X.shape)                 

/home/mgraca/miniconda3/envs/hrrrenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import os
from datetime import datetime

# Create output directory for results
output_dir = f"experiment_output/{EXPERIMENT_ID}"
os.makedirs(output_dir, exist_ok=True)

# Create dataset manager
manager = create_dataset_manager(
    registry_file=f"{EXPERIMENT_ID}_registry.json",
    cache_dir="data/jan_2025_with_hrrr/pwwb_cache/"
)

# List existing datasets
print("Existing datasets:")
try:
    display(manager.list_datasets())
except:
    print("No existing datasets found.")

Existing datasets:


,name,created,description,start_date,end_date,channels
0,jan_2025_with_hrrr_MAIAC_TROPOMI_NO2_METAR_WIN...,2025-06-13T19:24:10.770995,"January 2025 fire period with PWWB, Airnow, an...",2025-01-01-00,2025-02-01-00,"maiac, tropomi, metar"


In [5]:
# Adjust end date for AirNow
end_date_adj = pd.to_datetime(END_DATE) - pd.Timedelta(hours=1)

# Dataset name and description
dataset_name = f"{EXPERIMENT_ID}_MAIAC_TROPOMI_NO2_METAR_WIND_UV"
dataset_desc = f"{EXPERIMENT_DESCRIPTION} - MAIAC, TROPOMI NO2, METAR Wind U/V components"

# ========== 1. Load PWWB Data ==========
print(f"\nLoading PWWB data for {EXPERIMENT_NAME}...")

# Check if dataset already exists in the registry
if manager.get_dataset_info(dataset_name) is not None:
    print(f"Dataset '{dataset_name}' already exists, loading from cache...")
    pwwb_data = manager.load_dataset(dataset_name, PWWBData)
else:
    print(f"Dataset '{dataset_name}' not found, creating new one...")
    # Create the dataset with the specified channels
    pwwb_data = manager.create_dataset(
        name=dataset_name,
        description=dataset_desc,
        PWWBData_class=PWWBData,
        start_date=START_DATE,
        end_date=END_DATE,
        extent=extent,
        frames_per_sample=frames_per_sample,
        dim=dim,
        include_channels={
            'maiac': True,                     # Include MAIAC AOD
            'tropomi': ['TROPOMI_NO2'],        # Only include NO2 from TROPOMI
            'metar': ['METAR_Wind_U', 'METAR_Wind_V'],  # Only wind components from METAR
            'modis_fire': False,               # Exclude MODIS fire data
            'merra2': False                    # Exclude MERRA2 data
        },
        verbose=True,
        output_dir=output_dir
    )
    # Save the dataset
    pwwb_data.save_data()

# Get the data and channel info
X_pwwb = pwwb_data.data
channel_info = pwwb_data.get_channel_info()
print(f"✓ PWWB data shape: {X_pwwb.shape}")
print(f"  Channels: {channel_info['channel_names']}")

# ========== 2. Load AirNow Data ==========
print(f"\nLoading AirNow data for {EXPERIMENT_NAME}...")
airnow_data = AirNowData(
    start_date=START_DATE,
    end_date=end_date_adj,
    extent=extent,
    airnow_api_key=os.getenv('AIRNOW_API_KEY'),
    save_dir='data/jan_2025_with_hrrr/airnow_cache/airnow.json',
    processed_cache_dir='data/jan_2025_with_hrrr/airnow_cache/airnow_processed.npz',
    frames_per_sample=frames_per_sample,
    dim=dim,
    elevation_path="../../libs/inputs/elevation.npy",
    mask_path="../../libs/inputs/mask.npy",
    sensor_whitelist=['Simi Valley - Cochran Street', 'Reseda', 'Santa Clarita', 'North Holywood', 'Los Angeles - N. Main Street', 'Compton', 'Long Beach Signal Hill', 'Glendora - Laurel'],
    use_whitelist=True,
    use_mask=False,
    force_reprocess=False
)
X_airnow = airnow_data.data
Y_targets = airnow_data.target_stations
print(f"✓ AirNow data shape: {X_airnow.shape}")
if Y_targets is not None:
    print(f"  Target stations shape: {Y_targets.shape}")
    print(f"  Number of sensors: {len(airnow_data.air_sens_loc)}")
    print(f"  Sensor names: {airnow_data.sensor_names}")
else:
    print("  No target stations available")

#========== 3. Load HRRR Data ==========
print(f"\nLoading HRRR data for {EXPERIMENT_NAME}...")
hrrr_data = HRRRData(
    start_date=START_DATE,
    end_date=END_DATE,
    extent=extent,
    extent_name='la_region',
    product='COLMD',
    frames_per_sample=frames_per_sample,
    dim=dim,
    verbose=True,
    processed_cache_dir='data/jan_2025_with_hrrr/hrrr_cache/hrrr_processed.npz',
    chunk_cache_dir='data/jan_2025_with_hrrr/hrrr_cache/hrrr_chunks',
    sample_setting=2
)
X_hrrr = hrrr_data.data
print(f"✓ HRRR data shape: {X_hrrr.shape}")


Loading PWWB data for January 2025 Fire Dataset...
Dataset 'jan_2025_with_hrrr_MAIAC_TROPOMI_NO2_METAR_WIND_UV' already exists, loading from cache...
Using cache prefix: jan_2025_with_hrrr_MAIAC_TROPOMI_NO2_METAR_WIND_UV_
Initialized PWWBData with 744 hourly timestamps
Date range: 2025-01-01 00:00:00 to 2025-01-31 23:00:00
Channels included: ['maiac', 'tropomi', 'metar']
TROPOMI channels: ['TROPOMI_NO2']
METAR channels: ['METAR_Wind_U', 'METAR_Wind_V']
Processing MAIAC AOD data...
Loading cached data from data/jan_2025_with_hrrr/pwwb_cache/jan_2025_with_hrrr_MAIAC_TROPOMI_NO2_METAR_WIND_UV_maiac_aod_data.npy
Processing TROPOMI data...
Including TROPOMI channels: ['TROPOMI_NO2']
Loading cached data from data/jan_2025_with_hrrr/pwwb_cache/jan_2025_with_hrrr_MAIAC_TROPOMI_NO2_METAR_WIND_UV_tropomi_no2_data.npy
Processing METAR meteorological data...
Elevation data not found at inputs/elevation.npy. Using flat elevation.
Initialized MetarDataSource with 2 channels: ['METAR_Wind_U', 'METAR

In [6]:
del airnow_data
del pwwb_data
del hrrr_data

# ========== 4. Create Combined Dataset ==========
print(f"\nCreating combined dataset for {EXPERIMENT_NAME}...")

# First, check shapes and find minimum timesteps
print(f"Data shapes before combining:")
print(f"  PWWB: {X_pwwb.shape}")
print(f"  AirNow: {X_airnow.shape}")
print(f"  HRRR: {X_hrrr.shape}")
print(f"  Targets: {Y_targets.shape}")

# Find the minimum number of timesteps across all datasets
min_timesteps = min(X_pwwb.shape[0], X_airnow.shape[0], X_hrrr.shape[0], Y_targets.shape[0])
print(f"Minimum timesteps across all datasets: {min_timesteps}")

# Trim all datasets to the same number of timesteps
X_pwwb = X_pwwb[:min_timesteps]
X_airnow = X_airnow[:min_timesteps]
X_hrrr = X_hrrr[:min_timesteps]
Y_targets = Y_targets[:min_timesteps]  # Don't forget to trim Y_targets too!

print(f"Data shapes after trimming:")
print(f"  PWWB: {X_pwwb.shape}")
print(f"  AirNow: {X_airnow.shape}")
print(f"  HRRR: {X_hrrr.shape}")
print(f"  Y_targets: {Y_targets.shape}")

# Combine all data sources
X_combined = np.concatenate([X_pwwb, X_airnow, X_hrrr], axis=-1)
print(f"Combined data shape: {X_combined.shape}")

# Display the number of channels from each source
pwwb_channels = X_pwwb.shape[4]
airnow_channels = X_airnow.shape[4]
hrrr_channels = X_hrrr.shape[4]
total_channels = X_combined.shape[4]

print(f"Channel breakdown:")
print(f"  PWWB: {pwwb_channels} channels")
print(f"  AirNow: {airnow_channels} channels")
print(f"  HRRR: {hrrr_channels} channels")
print(f"  Total: {total_channels} channels")

# Create combined channel names
all_channel_names = channel_info['channel_names'] + ["AirNow_PM25"] + ["HRRR_COLMD"]
print(f"  Channel names: {all_channel_names}")


Creating combined dataset for January 2025 Fire Dataset...
Data shapes before combining:
  PWWB: (740, 5, 40, 40, 4)
  AirNow: (740, 5, 40, 40, 1)
  HRRR: (740, 5, 40, 40, 1)
  Targets: (735, 5, 8)
Minimum timesteps across all datasets: 735
Data shapes after trimming:
  PWWB: (735, 5, 40, 40, 4)
  AirNow: (735, 5, 40, 40, 1)
  HRRR: (735, 5, 40, 40, 1)
  Y_targets: (735, 5, 8)
Combined data shape: (735, 5, 40, 40, 6)
Channel breakdown:
  PWWB: 4 channels
  AirNow: 1 channels
  HRRR: 1 channels
  Total: 6 channels
  Channel names: ['MAIAC_AOD', 'TROPOMI_NO2', 'METAR_Wind_U', 'METAR_Wind_V', 'AirNow_PM25', 'HRRR_COLMD']


In [7]:
del X_pwwb
del X_airnow
del X_hrrr

print(f"Scaling test data for {EXPERIMENT_NAME} from the two years of training data...")
for i, c in enumerate(all_channel_names): 
    # ========== 6. Standardize Data ==========
    X_scaled = scale(
        X=X_combined[:,:,:,:,i], 
        scaler_path=f'data/two_years_with_hrrr/scalers/{c}_std_scaler.bin'
    )
    print(f"✓ Scaled test shape: {X_scaled.shape}")
    np.save(f"final_input_data/jan_2025_with_hrrr/{c}_X_jan_test.npy", X_scaled)
    print(f"Saved {c} data.\n")
    
np.save(f"final_input_data/jan_2025_with_hrrr/Y_jan_test.npy", Y_targets)

Scaling test data for January 2025 Fire Dataset from the two years of training data...
✓ Scaled test shape: (735, 5, 40, 40)
Saved MAIAC_AOD data.

✓ Scaled test shape: (735, 5, 40, 40)
Saved TROPOMI_NO2 data.

✓ Scaled test shape: (735, 5, 40, 40)
Saved METAR_Wind_U data.

✓ Scaled test shape: (735, 5, 40, 40)
Saved METAR_Wind_V data.

✓ Scaled test shape: (735, 5, 40, 40)
Saved AirNow_PM25 data.

✓ Scaled test shape: (735, 5, 40, 40)
Saved HRRR_COLMD data.

